In [1]:
# Código para Google Colab

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Descargar dataset de flores de TensorFlow (3670 imágenes, 5 clases)
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
path_to_zip = tf.keras.utils.get_file('flower_photos.tgz', origin=_URL, extract=True)
data_dir = path_to_zip.replace('flower_photos.tgz', 'flower_photos')

# Crear dataset de entrenamiento y validación
batch_size = 32
img_height = 180
img_width = 180

train_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Normalizar pixeles
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Cargar modelo base MobileNetV2 para transfer learning
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # congelar base para entrenar rápido

# Añadir capas para clasificación
model = models.Sequential([
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(5, activation='softmax')  # 5 clases de flores
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar
epochs = 5
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

# Guardar modelo
model.save('flower_classifier_model')


228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Found 3670 files belonging to 1 classes.
Using 2936 files for training.
Found 3670 files belonging to 1 classes.
Using 734 files for validation.


<ipython-input-1-240684509>:40: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.9404 - loss: 0.1755 - val_accuracy: 1.0000 - val_loss: 4.9595e-04
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 1.0000 - loss: 3.5560e-04 - val_accuracy: 1.0000 - val_loss: 4.0441e-04
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 1.0000 - loss: 2.9008e-04 - val_accuracy: 1.0000 - val_loss: 3.2505e-04
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 1.0000 - loss: 2.3014e-04 - val_accuracy: 1.0000 - val_loss: 2.6528e-04
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 1.0000 - loss: 1.7527e-04 - val_accuracy: 1.0000 - val_loss: 2.1913e-04


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=flower_classifier_model.